|평가문항|상세기준|
|:--------|:--------|
|1. 한국어 전처리 과정이 적절하였는가?|형태소 분석기 선택과 불용어 제거가 체계적으로 진행됨|
|2. 텍스트 데이터 수집이 분량과 다양성 측면에서 적절했는가?|일자와 분량에서 텍스트 데이터 다양성 향상을 위한 노력이 확인됨|
|3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?|F-1 score 기준 83% 이상의 정확도가 확인됨|

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from newspaper import Article

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [3]:
### 사용자 입력 처리
print("페이지 수를 지정해 주세요.")
input_page = input("페이지수: ")
page_num = int(input_page)

print("카테고리 코드를 입력해 주세요. (예: 100, 101, 102)")
print("100:정치, 101:경제, 102:사회, 103:생활/문화, 104:세계, 105:IT/과학")
input_code = input("카테고리 코드: ")
code_list = [int(code.strip()) for code in input_code.split(',')]

print("크롤링을 시작할 날짜를 00000000형식으로 지정하세요.")
input_sdate = input("크롤링 시작 날짜:")

print("크롤링을 마칠 날짜를 00000000형식으로 지정하세요.")
input_edate = input("크롤링 종료 날짜:")

페이지 수를 지정해 주세요.
페이지수: 2
카테고리 코드를 입력해 주세요. (예: 100, 101, 102)
100:정치, 101:경제, 102:사회, 103:생활/문화, 104:세계, 105:IT/과학
카테고리 코드: 102, 105
크롤링을 시작할 날짜를 00000000형식으로 지정하세요.
크롤링 시작 날짜:20231231
크롤링을 마칠 날짜를 00000000형식으로 지정하세요.
크롤링 종료 날짜:20240101


In [4]:
### 날짜 데이터 분리
def parse_date(date_str):
    year = date_str[:4]
    month = date_str[4:6]
    day = date_str[6:]
    formatted_date = f"{year}-{month}-{day}"
    return datetime.strptime(formatted_date, '%Y-%m-%d')

start_date = parse_date(input_sdate)
end_date = parse_date(input_edate)


### 시작 일자부터 종료 일자까지의 날짜를 리스트로 생성
date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=1)

print(f"{start_date.strftime('%Y-%m-%d')}부터 {end_date.strftime('%Y-%m-%d')}까지의 데이터를 크롤링합니다.")

int_date_list = [int(date.replace('-', '')) for date in date_list]


2023-12-31부터 2024-01-01까지의 데이터를 크롤링합니다.


In [5]:
# 페이지 수, 카테고리, 날짜를 입력값으로 받습니다.
def make_urllist(page_num, code, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)

    # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [ ]:
# 모든 URL을 저장할 리스트
all_urllist = []

# 각 카테고리에 대한 데이터프레임을 저장할 변수 초기화
df = pd.DataFrame()
def make_data(urllist, code):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        text_list.append(article.title)
    df = pd.DataFrame({'news': text_list})
    df['code'] = idx2word[str(code)]
    return df

# 코드로부터 바로 어떤 카테고리인지 확인하기 쉽도록 딕셔너리 생성
idx2word = {'100': '정치', '101': '경제', '102': '사회', '103': '생활/문화', '104': '세계', '105': 'IT/과학'}




for code in code_list:
    code_urllist = []  # 각 카테고리에 대한 URL 리스트

    for date in int_date_list:
        urllist = make_urllist(page_num, code, date)
        code_urllist.extend(urllist)  # 각 날짜별 URL을 code_urllist에 추가

    all_urllist.extend(code_urllist)  # 각 카테고리별 URL 리스트를 all_urllist에 추가
    df_temp = make_data(code_urllist, code)
    print(str(code) + '번 코드에 대한 데이터를 만들었습니다.')

    df = pd.concat([df, df_temp])  # 각 카테고리별 데이터프레임을 df에 추가

print("총", len(all_urllist), "개의 기사를 수집했습니다.")

In [ ]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

In [ ]:
# 결측치 확인
print(df.isnull().sum())

In [ ]:
# 중복된 샘플들을 제거
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

In [ ]:
from konlpy.tag import Hannanum

# Hannanum 형태소 분석기 초기화
tokenizer = Hannanum()

# 불용어 리스트 정의
stopwords = ['에', '는', '을', '를', '이', '가', '도', '다', '의', '한', '과', '와', '으로', '하다', ]

# 데이터 전처리 및 토큰화 함수
def preprocessing(data):
    text_data = []

    for sentence in data:
        temp_data = tokenizer.morphs(sentence)  # 토큰화
        temp_data = [word for word in temp_data if not word in stopwords]  # 불용어 제거
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

# 샘플 출력
text_data = preprocessing(df['news'])
print(text_data[:5])

In [ ]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [ ]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

In [ ]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [ ]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

마지막에 커널과의 연결이 끊겨서 output 데이터를 출력하지 못했는데 IT/과학쪽은 f-score가 0.75였습니다
생각보다 구현이 간단한 듯 복잡하네요
더 많은 공부가 필요할 것 같습니다...